In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import GPyOpt
import GPy 
import pickle

import RPE_normalize
import importlib
importlib.reload(RPE_normalize)

from RPE_normalize import X_names, X_bounds, get_domain
from RPE_normalize import get_batch_context, get_init_design, rescale

In [2]:
normalize = True
dfopen =  pd.read_csv("./r2results_opening.csv", index_col='name')
#dfopen =  pd.read_csv("./r2results_sum.csv", index_col='name')
results = dfopen.values
Y_init = []
for plate in range(results.shape[1]):
    for well in range(results.shape[0]):
        Y_init.append([results[well][plate]])
Y_init = -1*np.array(Y_init)
X_init = get_init_design(normalize=normalize)

dfopen

,Plate1,Plate2,Plate3,Plate4,Plate5,Plate6,Plate7,Plate8
name,,,,,,,,
Well1,0.006800,0.002890,0.000243,0.218000,0.000144,0.000054,0.000074,0.162000
Well2,0.000293,0.000102,0.000120,0.498000,0.001540,0.000040,0.000000,0.476000
Well3,0.004180,0.856000,0.000324,0.000231,0.001080,0.865000,0.000037,0.000616
Well4,0.128000,0.000096,0.000113,0.203000,0.122000,0.000050,0.000023,0.210000
Well5,0.000448,0.470000,0.000567,0.000062,0.003700,0.709000,0.000022,0.000000
Well6,0.047600,0.000508,0.000041,0.055400,0.072000,0.000446,0.000008,0.025400


In [3]:
savepath= "./trypsin28_43_precon1_6mat52_init1"

jorders = [0]
for i in np.arange(0,10):
    aaa = 
    jorders.append(float(aaa))

print(jorders)

In [4]:
jorders = [0,1e-2,5e-3,2e-3,1e-3,5e-4,1e-4,5e-5,1e-5]

jorders = [0,2e-18,3e-18,4e-18,5e-18,6e-18,7e-18,8e-18,9e-18,]

In [5]:
def calc_EIstep(X_init, Y_init, jitter,batch_context=True,normalize = normalize, savepath = savepath):
    space = GPyOpt.core.task.space.Design_space(get_domain(normalize=normalize), None)
    #model_gp = GPyOpt.models.GPModel(
    #    kernel=GPy.kern.RBF(input_dim=7, ARD=True),ARD=True, verbose=False)
    model_gp = GPyOpt.models.GPModel(ARD=True, verbose=False)
    objective = GPyOpt.core.task.SingleObjective(None)
    acquisition_optimizer = GPyOpt.optimization.AcquisitionOptimizer(space)
    acquisition_EI = GPyOpt.acquisitions.AcquisitionEI(model_gp, space, acquisition_optimizer, jitter=jitter)
    acquisition = GPyOpt.acquisitions.LP.AcquisitionLP(model_gp, space, acquisition_optimizer,acquisition_EI)
    evaluator = GPyOpt.core.evaluators.LocalPenalization(acquisition, batch_size=48)
    
    bo_EI = GPyOpt.methods.ModularBayesianOptimization(
    model=model_gp,
    space=space,
    objective=objective,
    acquisition=acquisition,
    evaluator=evaluator,
    X_init=X_init,
    Y_init=Y_init,   
    normalize_Y=True
    )
    if batch_context:
        nextX = bo_EI.suggest_next_locations(batch_context=get_batch_context(normalize=normalize))
    else:
        nextX = bo_EI.suggest_next_locations()
        
    if normalize:
        nextX = rescale(nextX)
    
    with open( savepath+"/model/EI_j"+str(jitter)+".pkl", "wb") as f:
        pickle.dump(bo_EI, f, protocol=2)
    
    return nextX

In [6]:
def x2csv(nextX, jitter,savepath = savepath):
    wells = np.array([])
    dishes = np.array([])
    for dishno in np.arange(1,9):
        wells = np.append(wells,[1,2,3,4,5,6])
        dishes = np.append(dishes,np.array([1,1,1,1,1,1])*dishno)
    wells = np.array([wells]).T
    dishes = np.array([dishes]).T
    test = np.concatenate((dishes,wells),axis=1)
    result = np.concatenate((test,nextX),axis=1)
    #result = pd.DataFrame(result, columns=["Dish_No","Well_No","細胞剥離方法","trypsin_time","FGFRi濃度","preconditioning_period","suspend速度","KSR期間","3因子期間"])
    result = pd.DataFrame(result, columns=["Dish_No","Well_No","cell_peelingmode","trypsin_time","FGFRi_conc","preconditioning_period","suspend_speed","KSR_period","3factor_period"])
    result.to_csv(savepath+"/csv/EI_j"+str(jitter)+".csv")
    return 0

In [7]:
for jitter in jorders:
    print(jitter)
    nextX = calc_EIstep(X_init, Y_init, jitter, batch_context=True, normalize = normalize)
    x2csv(nextX, jitter,savepath= savepath )

0
0.01
0.005
0.002
0.001
0.0005
0.0001
5e-05
1e-05
